In [ ]:
import os
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

In [ ]:
#1. understand the problem 
#2.decide which model to use 
#3.messurement matrix 
#4.extra about collabe
#5.load data if you want to do it quit ofter just make a function for it i am not doing since it's already here. 

def load_housing_data():
    tar_file=Path('datasets/housing.tgz')
    return pd.read_csv(Path('datasets/housing/housing.csv'))

In [ ]:
housing = load_housing_data()

In [ ]:
housing.head()

In [ ]:
#let's take a quick look at data how it organize and where to deal with it.
                                
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
import matplotlib.pyplot as plt

housing.hist(bins=50,figsize=(14,8))
plt.show()

In [ ]:
#6 TRAIN TEST SPLIT
# it's too early to use train test split but for the better understanding of data we have done this eralier ,
# also we have done random split but in order to not introduct bias we have split strtigically
#there are many ways to do this so we are exploring it in many cells.
from sklearn.model_selection import train_test_split

In [ ]:
train_set,test_set = train_test_split(housing,test_size=0.2,random_state=42)

In [ ]:
print(len(train_set))
print(len(test_set))

In [ ]:
#zooming on median_income column with histogram.
import numpy as np
housing['median_income'].plot(kind='hist',bins=29,figsize=(12,6))
labels=np.linspace(0,14,29).round(1)
plt.xticks(labels)
plt.show()

In [ ]:
#using pd.cut() func to convert income column into categorical column.
housing['income_cat']=pd.cut(housing['median_income'],bins=[0,1.5,3,4.5,6,np.infty],labels=['0-1.5','1.5-3','3-4.5','4.5-6','6-\u221E'])

In [ ]:
housing['income_cat'].value_counts().sort_index().plot(kind='bar',rot=0,grid=True)

In [ ]:
#now let's create more than two sets with same ratio. but random elements in each sets. also it devide same portion of the lable column or
# column you want to. 
from sklearn.model_selection import StratifiedShuffleSplit
spliter=StratifiedShuffleSplit(n_splits=10,test_size=0.2,random_state=42)
list=[]
for train_index,test_index in spliter.split(housing,housing['income_cat']):
    # print('train index :',train_index)
    # print('test index :',test_index)
    train_set_n=housing.iloc[train_index]
    test_set_n=housing.iloc[test_index]
    # print('train set :',train_set)
    # print('test set :',test_set)
    list.append([train_set,test_set])

train_set_1,test_set_1=list[0]
# print(train_set_1)
# print('test set' , test_set_1)

In [ ]:
# if you just want 1 set train test split will do the job for you 
train_set_n,test_set_n=train_test_split(housing,test_size=0.2,random_state=42,stratify=housing['income_cat'])

In [ ]:
print('overall proportion of income_cat column :\n',(housing['income_cat'].value_counts().sort_index())/len(housing))
print('\ntrainset proportion of income_cat column :\n',(train_set_n['income_cat'].value_counts().sort_index())/len(train_set_n))
print('\ntestset proportion of income_cat column :\n',(test_set_n['income_cat'].value_counts().sort_index())/len(test_set_n))

In [ ]:
#we just created income_cat column to do this so let's drop it 
housing=housing.drop('income_cat',axis=1)
train_set_n=train_set_n.drop('income_cat',axis=1)
test_set_n=test_set_n.drop('income_cat',axis=1)

In [ ]:
housing=train_set_n.copy()

In [ ]:
housing['population'].describe()

In [ ]:
#EXPLORATION OF DATA
#now we really exploring the data and labels
housing.plot(kind='scatter',x='longitude',y='latitude',alpha=0.5,s=housing['population']/10,label='population',colorbar=True,cmap='jet',
            c=housing['median_house_value'],edgecolors='red',figsize=(20,10),legend=True)
plt.show()

In [ ]:
#it's a nice thing to see the corr between labels and other features
coormat=housing.corr(numeric_only=True)

In [ ]:
coormat['median_house_value']


In [ ]:
#now let's visulized the correlation using scatter_matrix func from pandas
# housing.info()
from pandas.plotting import scatter_matrix
plot_col=['housing_median_age','total_rooms','median_income','median_house_value']
scatter_matrix(housing[plot_col],figsize=(12,8))
plt.show()

In [ ]:
#since there is only one feature meadian income seems to relete the house value let's plot a scatter plot for it 
housing.plot(kind='scatter',x='median_income',y='median_house_value',alpha=0.4)
plt.yticks(np.linspace(0,500000,11))
plt.show()

In [ ]:
# as we can see in our df the rooms , bedrooms , poplution given based on district but if it's in per person it will make more sence.
housing.head()
housing['rooms_per_house']=housing['total_rooms']/housing['households']
housing['bedroom_ratio']=housing['total_bedrooms']/housing['total_rooms']
housing['people_per_house']=housing['population']/housing['households']

In [ ]:
housing.corr(numeric_only=True)['median_house_value']

In [ ]:
#now let's create x and y for model
housing_lable=train_set_n['median_house_value']
housing=train_set_n.drop('median_house_value',axis=1)

In [ ]:
housing.info()

In [ ]:
#HANDLING WITH MISSING DATA
#as you can see there are some missing values in total bedrooms column you can get rid of it using three methods of pandas 
# 1. drop the column containg missing values 
housing.drop('total_bedrooms',axis=1)
#2. remove whole row which cointain missing values for total bedrooms
housing.dropna(subset=['total_bedrooms'],inplace=True)
housing.info()
#3. this both methods at the end removing data from the df so we should be go with fill missing values with the median
housing=train_set_n.drop('median_house_value',axis=1)
median=housing['total_bedrooms'].median()
housing['total_bedrooms'].fillna(median,inplace=True)
housing.info()

In [ ]:
#but it is more mannual way and scarry too. so you can use the simpleimputer from sklearn to fill missing values in whole df by a method you want to 

from sklearn.impute import SimpleImputer
#creating a instace of imputer
imp = SimpleImputer(strategy='median')

#imputer only work on numberical data so excluding the other datatype you can also use include argument with 'number'
housing_num=housing.select_dtypes(exclude='object')
housing_num.info()

#then simply fit that df and it will generate the median for each column
imp.fit(housing_num)
print('median using imputer :\n',imp.statistics_)
print('median using numpy on df :\n',housing_num.median().values)


In [ ]:
X=imp.transform(housing_num)
housing_tr=pd.DataFrame(X,columns=housing_num.columns,index=housing_num.index)
housing_tr.info()

In [ ]:
#HANDLING TEXT AND CATEGORICAL DATA
#we are gone use ordinal encoder which convert this categories into values.
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
housing_cat=ordinal_encoder.fit_transform(train_set_n[['ocean_proximity']])
housing_cat

In [ ]:
# as you can see it's numbers are not releted here but model will recognized as releted so we use onehotencode to transform this into 0-1
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder=OneHotEncoder()
housing_cat_1hot=one_hot_encoder.fit_transform(housing_cat)
# print(housing_cat_1hot)
housing_cat_1hot.toarray()

In [ ]:
#well it was a array which is not much meaning full so let's convert this array into dataframe.
one_hot_encoder = OneHotEncoder(sparse_output=False)
housing_cat_arr=one_hot_encoder.fit_transform(train_set_n[['ocean_proximity']])
housing_cat_df = pd.DataFrame(housing_cat_arr,columns=one_hot_encoder.get_feature_names_out(),index=train_set_n.index)
housing_cat_df.head()

In [ ]:
# FEATURE SCALING AND TRASFORMATION 
# 1.using minmaxscaler
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler(feature_range=(-1,1))
housing_num_minmax_scale=minmaxscaler.fit_transform(housing_num)

# 2.using standrd scaler
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
housing_num_standard_scale=standardscaler.fit_transform(housing_num)

In [ ]:
# tailed distribution - both method failed because both method will distroy the outlier or model predictions
# Solution - try to make distribution symetric 
# Ways to do that - 1. positive features with tail to right -- replace feature with it's values of power between 0 to 1 ex (sqrt) 
# 2. heavy tail -- try to use log scale 
# 3. devide the values in bucket just like we did when create income category column 
# 4. multimodel distribution ⅰ.using bucket but must use OneHotIncoder 
# ⅱ. add a feature for each mode - achieving using radial basis function (RBF) 
# most comman is gaussian RBF : exp(-γ(x-35)²) -- as the values goes far from 35 it will decrease fast.

from sklearn.metrics.pairwise import rbf_kernel
housing_age_rbf_35=rbf_kernel(train_set_n[['housing_median_age']],[[35]],gamma=.1)

In [ ]:
# since we are studying the transformation if we transform the label data how we understand the real values here inverse transform func come to play.
scaler =StandardScaler()
labels=scaler.fit_transform(housing_lable.to_frame())
# print(labels)
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(housing[['median_income']],labels)

pred = model.predict(housing[['median_income']][:5])
scaler.inverse_transform(pred)

In [ ]:
# well we can do all the things just by one function not need to go this long way 

from sklearn.compose import TransformedTargetRegressor

model =TransformedTargetRegressor(LinearRegression(),transformer=StandardScaler())
model.fit(housing[['median_income']],housing_lable)

model.predict(housing[['median_income']][:5])

In [ ]:
# Now let's see how you can create you own tranfrom func. Many time you needs to create you own transformation func for various transformation
from sklearn.preprocessing import FunctionTransformer

#example of use case of Function Trasformer
log_transformer = FunctionTransformer(np.log,np.exp)
log_values = log_transformer.transform(housing['housing_median_age'])
# print(housing['housing_median_age'])
log_values

ratio_transformer = FunctionTransformer(lambda X: X[:, [0]] / X[:, [1]])
ratio_transformer.transform(np.array([[1., 2.], [3., 4.]]))

#you can also create new transfomer from a old transformer and give parameter of the old one in new as a fix values 
rbf_transformer = FunctionTransformer(rbf_kernel,
kw_args=dict(Y=[[35.]], gamma=0.1))
age_simil_35 = rbf_transformer.transform(housing[["housing_median_age"]])

# 

In [ ]:
# We just see how to use transform , but what if we also want to use fit and trasform both function here by inherting a class we can do that.

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.base import check_array,check_is_fitted

class StandardScalerClone(BaseEstimator,TransformerMixin):
    def __init__(self,with_mean=True):
        self.with_mean=with_mean

    def fit(self,X,y=None):
        X=check_array(X)
        self.mean_=X.mean()
        self.std_=X.std()
        self.n_features_in_=X.shape[1]
        return self
        
    def transform(self,X):
        check_is_fitted(self)
        X=check_array(X)
        assert self.n_features_in_==X.shape[1]
        if self.with_mean:
            X=X-self.mean_
        return X/self.std_

#well the thing to note here is we don't use func like : inverse_transform(),get_feature_names_in(),get_features_name_out() but we have to 
# whenever we build own transformer. 

In [ ]:
stdscaler=StandardScalerClone()
stdscaler.fit(housing_num)

In [ ]:
stdscaler.n_features_in_

In [ ]:
stdscaler.transform(housing_num)

In [ ]:
#Also custom trasformer can use other estimator 
from sklearn.cluster import KMeans

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state
    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self # always return self!
    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

In [ ]:
#PIPELINE
# as you can see you have to do many transformation in right order here pipeline comes to play
import sklearn as sklearn
sklearn.set_config(display="diagram")
from sklearn.pipeline import Pipeline

imp_stdscaler_pip=Pipeline([('imputer',SimpleImputer(strategy='median')),('scaler',StandardScaler())])

# things to note 1.name can be anything just two limits --unique names --not __ (double underscore)
# 2.in pipeline you can pass as many as arguments you want but all must be trasfomer except last one it can be anything(predictor,estimator..)

imp_stdscaler_pip

In [ ]:
# However if you don't want to use the names you can use make_pipeline() function 
# just using .fit() you can do fit and transform.

from sklearn.pipeline import make_pipeline

imp_stdscaler_pip2=make_pipeline(SimpleImputer(strategy='median'),StandardScaler())
imp_stdscaler_pip2

#if the last method is predictor you use .predict() - do all trasformation and predict.
#if last method is trasfomer you use .transform() - do all transformation.

In [ ]:
# pipline also have get_feature_names_out() func which is usefull for creating df out of array.
housing_tr_arr=imp_stdscaler_pip.fit_transform(housing_num)
housing_tr_df=pd.DataFrame(housing_tr_arr,columns=imp_stdscaler_pip.get_feature_names_out(),index=housing_num.index)
housing_tr_df

In [ ]:
#pipeline also support indexing
imp_stdscaler_pip[:-1]
imp_stdscaler_pip.named_steps

In [ ]:
# Now as we have seen we have to handle cat and num columns seprate but what if some pipeline do that 
cat_pip=make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder())

housing=pd.read_csv('datasets/housing/housing.csv')
num_col=housing.columns[:-1]
cat_col=housing.columns[-1]

from sklearn.compose import ColumnTransformer

transformer=ColumnTransformer([('num_pip',imp_stdscaler_pip,num_col),('cat_pip',cat_pip,cat_col)])
transformer

In [ ]:
# there is still some function we can use instead of doing things manually
from sklearn.compose import make_column_selector,make_column_transformer

transformer1=make_column_transformer((imp_stdscaler_pip,make_column_selector(dtype_exclude='object')),
                                    (cat_pip,make_column_selector(dtype_include='object')))

housing_tr1=transformer1.fit_transform(housing)

In [ ]:
# So far we have seen how to use transformer and pipeline now let's make some functions to do this repeted things 

def column_ratio(X):
    return X[:,[0]]/X[:,[1]]

def ratio_name(function_transformer,feature_names_in):
    return ['ratio']

def ratio_pipeline():
    return make_pipeline(SimpleImputer(strategy='median'),
                         FunctionTransformer(column_ratio,feature_names_out=ratio_name),
                         StandardScaler())

log_pipeline=make_pipeline(SimpleImputer(strategy='median'),
                           FunctionTransformer(np.log,feature_names_out='one-to-one'),
                           StandardScaler())

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)
default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
StandardScaler())

processing=ColumnTransformer([
("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
("people_per_house", ratio_pipeline(), ["population", "households"]),
("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
"households", "median_income"]),
("geo", cluster_simil, ["latitude", "longitude"]),
("cat", cat_pip, make_column_selector(dtype_include=object)),
],
remainder=default_num_pipeline)


In [ ]:
housing_final=processing.fit_transform(housing)

In [ ]:
processing.get_feature_names_out()

In [ ]:
y=train_set_n['median_house_value']
train_set_n.drop('median_house_value',axis=1,inplace=True)

In [ ]:
#SELECT AND TRAIN A MODEL 
from sklearn.linear_model import LinearRegression

housing_lable=housing['median_house_value']
lin_pip = make_pipeline(processing,LinearRegression())
lin_pip.fit(train_set_n,y)

In [ ]:
# y_origional=test_set_n['median_house_value']
# test_set_n.drop('median_house_value',inplace=True,axis=1)
pred = lin_pip.predict(train_set_n)
pred

In [ ]:
housing_lable

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y,pred,squared=False)

In [ ]:
# as you can see the error is very high so you have to use other model to see which works better on this data

from sklearn.tree import DecisionTreeRegressor

dec_pip=make_pipeline(processing,DecisionTreeRegressor(random_state=42))

dec_pip.fit(train_set_n,y)

In [ ]:
pred=dec_pip.predict(train_set_n)
mean_squared_error(y,pred,squared=False)

In [ ]:
# we have use whole data to predict the model but it is better to use train test split and even better cross validation 
from sklearn.model_selection import cross_val_score

cross_validation_dic=-cross_val_score(dec_pip,train_set_n,y,scoring="neg_root_mean_squared_error",cv=10)

In [ ]:
pd.Series(cross_validation_dic).describe()

In [ ]:
# as you can see it perform same as linear regression model now let's try random forest model and see how it works on data 

from sklearn.ensemble import RandomForestRegressor

ran_pip=make_pipeline(processing,RandomForestRegressor(random_state=42))

cross_validation_ran=-cross_val_score(ran_pip,train_set_n,y,scoring='neg_root_mean_squared_error',cv=10)
cross_validation_ran

In [ ]:
pd.Series(cross_validation_ran).describe()

In [ ]:
# GRID SEARCH CV - use for selecting best parameters for models AKA fine tuning 
from sklearn.model_selection import GridSearchCV

ran_grid_pip=Pipeline([('preprocessing',processing),('random_forest',RandomForestRegressor(random_state=42))])

param_grid = [
{'preprocessing__geo__n_clusters': [5, 8, 10],
'random_forest__max_features': [4, 6, 8]},
{'preprocessing__geo__n_clusters': [10, 15],
'random_forest__max_features': [6, 8, 10]},
]

grid_search=GridSearchCV(ran_grid_pip,param_grid=param_grid,scoring='neg_root_mean_squared_error',cv=3)
grid_search.fit(train_set_n,y)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_result=pd.DataFrame(grid_search.cv_results_)
grid_result

grid_result.sort_values(by='mean_test_score',ascending=False).head()

In [ ]:
# Random search is usefull when you have many parameter to test it selects random combo . think when you have continues values it help there.

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'preprocessing__geo__n_clusters': randint(low=3, high=50),
'random_forest__max_features': randint(low=2, high=20)}

random_search=RandomizedSearchCV(ran_grid_pip,param_distribs,scoring='neg_root_mean_squared_error',cv=3,random_state=42)
random_search.fit(train_set_n,y)

In [ ]:
pd.DataFrame(random_search.cv_results_).sort_values('mean_test_score',ascending=False).head()

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV,HalvingRandomSearchCV
#this both are use for more exploration of hyperparameter or for using better computation power

In [ ]:
#chosing the best model exploring the model results 
final_model=random_search.best_estimator_

In [ ]:
# you can see which feature is how much imporatant
feature_imp=(final_model["random_forest"].feature_importances_).round(2)
feature_imp

In [ ]:
#you can get correspoding features name
final_model_feature_names=final_model['preprocessing'].get_feature_names_out()
pd.DataFrame(data=feature_imp,index=final_model_feature_names).sort_values(by=0,ascending=False)

In [ ]:
# evaluting the model on test set 

X_test=test_set_n.drop('median_house_value',axis=1)
y_test=test_set_n['median_house_value']

y_pred=final_model.predict(X_test)
mean_squared_error(y_test,y_pred,squared=False)

In [ ]:
# another way to messure error in more details 
from scipy import stats
confidence = 0.95
squared_errors = (y_pred - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
loc=squared_errors.mean(),
scale=stats.sem(squared_errors)))

In [ ]:
# LAUNCH MONITER AND MAINTAIN YOUR SYSTEM 

#saving the model 
import joblib

joblib.dump(final_model,'Regresion_model_predict_house_price.pkl')

In [1]:
# after this you can deploy model and make sure to maintain it.
# chapter-2 done 